In [2]:
from hashlib import new
from django.db import models
from unicodedata import name
from finvizfinance.screener.overview import Overview
from finvizfinance.screener.valuation import Valuation
from finvizfinance.quote import finvizfinance
import math
import statistics
import time
from urllib.request import Request, urlopen
import regex as re
#from django.apps import AppConfig
from django.conf import settings
from logging.config import IDENTIFIER
from audioop import avg
from pickle import TRUE
from cmath import nan
import pandas as pd

DOLLAR_MAPPING = {
    'T': float(1000000000000),
    'B': float(1000000000),
    'M': float(1000000),
    'K': float(1000),
    'e': float(0)
}
# Helps create floats from shortened long numbers
def fix_dashes(x):
        if x == '-':
            x = 0
            return x
        else:
            x = round(float(x),2)
            return x
    # Gets rid of the dash for - values
stock_dic = {}

def get_stock_dic(ind):
    industry = ind
    foverview = Valuation()
    filters_dict = {'Industry': industry}
    foverview.set_filter(filters_dict=filters_dict)
    df = foverview.screener_view()
    time.sleep(1)
    ticker_list = df['Ticker']
    for stock_ticker in ticker_list:
        stock = None
        while stock == None:
            try:
                stock = finvizfinance(stock_ticker).ticker_fundament()
                time.sleep(.5)
            except Exception:
                time.sleep(5)
    # Stores the basic stock data as a variable
        sector = stock['Sector']
        # Gets the sector
        ticker = stock_ticker
        # Gets the ticker
        company_name = stock['Company']
        # Gets the company name
        beta = stock['Beta']
        beta = fix_dashes(beta)
        pb = stock['P/B']
        pb = fix_dashes(pb)
        ps = stock['P/S']
        ps = fix_dashes(ps)
        pe = stock['P/E']
        pe = fix_dashes(pe)
        fwdpe = stock['Forward P/E']
        fwdpe = fix_dashes(fwdpe)
        eps = stock['EPS (ttm)']
        eps = fix_dashes(eps)
        mkt_cap_short = stock['Market Cap']
        if mkt_cap_short == '-':
            mkt_cap_short = str(0)
            mkt_cap = 0
        else:
            mkt_cap = float(mkt_cap_short[:-1]) * float(DOLLAR_MAPPING[mkt_cap_short[-1]])
        #gets mkt cap
        revenue_short = stock['Sales']
        if revenue_short == '-':
            revenue_short = str(0)
            revenue = 0
        else:
            revenue = float(revenue_short[:-1]) * float(DOLLAR_MAPPING[revenue_short[-1]])
        # gets the sales
        profit_short = stock['Income']
        if profit_short == '-':
            profit = 0
            profit_short = str(0)
        else:
            profit = float(profit_short[:-1]) * float(DOLLAR_MAPPING[profit_short[-1]])
        # gets the net income
        profit_margin = stock['Profit Margin']
        if profit_margin == '-':
            profit_margin = str(0)
            profit_margin_float = 0
        else:
            profit_margin_float = float(profit_margin[:-1])/100
        # gets the PM
        rev_growth = stock['Sales Q/Q']
        if rev_growth == '-':
            rev_growth = str(0)
            rev_growth_float = 0
        else:
            rev_growth_float = float(rev_growth[:-1])/100
        # gets rev growth
        avg_volume = stock['Avg Volume']
        shares_float = stock['Shs Float']
        short_float = stock['Short Float']
        # gets revenue growth
        stock_dic[ticker] = {'company_name': company_name, 'ticker': ticker, 'industry': industry, 'beta': beta, 'mkt_cap' : mkt_cap,'mkt_cap_short': mkt_cap_short, 'revenue': revenue, 'revenue_short':revenue_short,'profit': profit, 'profit_margin':profit_margin, 'sector': sector,'rev_growth': rev_growth, 'pe': pe,'fwdpe':fwdpe, 'profit_margin_float': profit_margin_float,'rev_growth_float':rev_growth_float,'ps':ps,'eps':eps,'pb':pb,'avg_volume':avg_volume,'shares_float':shares_float,'short_float':short_float}
    return stock_dic

def get_industry_list():
    req = Request("https://finviz.com/groups.ashx?g=industry&v=110&#o=name",headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read().decode('utf-8')
    industries = re.findall('(?<=class="tab-link">).*?(?=</a>)',webpage,re.DOTALL)
    industries = industries[:144]
    big_industries = [industries[13],industries[17],industries[118],industries[8],industries[9],industries[37],industries[40],industries[43],industries[44],industries[62],industries[120],industries[121]]
    small_industries = []
    for industry in industries:
        if industry not in big_industries:
            small_industries.append(industry)
    return small_industries

def get_stock_info():
    industries = get_industry_list()
    industries = industries[:1]
    for industry in industries:
        get_stock_dic(industry)
        for stock in stock_dic:
            print(stock,stock_dic[stock])

get_stock_info()

ADTH {'company_name': 'AdTheorent Holding Company, Inc.', 'ticker': 'ADTH', 'industry': 'Advertising Agencies', 'beta': 0, 'mkt_cap': 176430000.0, 'mkt_cap_short': '176.43M', 'revenue': 171200000.0, 'revenue_short': '171.20M', 'profit': 0, 'profit_margin': '0', 'sector': 'Communication Services', 'rev_growth': '6.50%', 'pe': 4.34, 'fwdpe': 20.76, 'profit_margin_float': 0, 'rev_growth_float': 0.065, 'ps': 1.03, 'eps': 0.44, 'pb': 1.2, 'avg_volume': '357.30K', 'shares_float': '79.39M', 'short_float': '1.46%'}
ADTHW {'company_name': 'AdTheorent Holding Company, Inc.', 'ticker': 'ADTHW', 'industry': 'Advertising Agencies', 'beta': 0, 'mkt_cap': 0, 'mkt_cap_short': '0', 'revenue': 157470000.0, 'revenue_short': '157.47M', 'profit': 0, 'profit_margin': '0', 'sector': 'Communication Services', 'rev_growth': '36.30%', 'pe': 0.61, 'fwdpe': 0, 'profit_margin_float': 0, 'rev_growth_float': 0.363, 'ps': 0, 'eps': 0.31, 'pb': 0.12, 'avg_volume': '11.65K', 'shares_float': '20.85M', 'short_float': '-'

In [3]:
aple = finvizfinance('AAPL').ticker_fundament()
print(aple['Income'])

99.63B


In [27]:
from finvizfinance.quote import finvizfinance

DOLLAR_MAPPING = {
    'T': float(1000000000000),
    'B': float(1000000000),
    'M': float(1000000),
    'K': float(1000),
    'e': float(0)
}

def human_format(num):
        num = float('{:.3}'.format(num))
        magnitude = 0
        while abs(num) >= 1000:
            magnitude += 1
            num /= 1000.0
        return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

stock = finvizfinance('ADNT')

stock_fundament = stock.ticker_fundament()

mkt_cap = float(stock_fundament['Market Cap'][:-1]) * float(DOLLAR_MAPPING[stock_fundament['Market Cap'][-1]])

book_sh = float(stock_fundament['Book/sh'])

shares_outstanding = float(stock_fundament['Shs Outstand'][:-1]) * float(DOLLAR_MAPPING[stock_fundament['Shs Outstand'][-1]])

book_value = human_format(book_sh * shares_outstanding)

print(book_value)

share_price = stock_fundament['Prev Close']

2.12B


In [21]:


req = Request('https://www.sec.gov/edgar/search/#/q='+text+'&filter_forms=10-Q',headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read().decode('utf-8')
    industries = re.findall('(?<=class="tab-link">).*?(?=</a>)',webpage,re.DOTALL)

AttributeError: type object 'finvizfinance' has no attribute 'get_statements'

In [28]:
print(stock_fundament)

{'Company': 'Adient plc', 'Sector': 'Consumer Cyclical', 'Industry': 'Auto Parts', 'Country': 'Ireland', 'Index': '-', 'P/E': '3.52', 'EPS (ttm)': '8.26', 'Insider Own': '0.90%', 'Shs Outstand': '94.80M', 'Perf Week': '-9.24%', 'Market Cap': '2.82B', 'Forward P/E': '8.95', 'EPS next Y': '2563.93%', 'Insider Trans': '-1.38%', 'Shs Float': '94.15M', 'Perf Month': '-13.79%', 'Income': '795.00M', 'PEG': '0.10', 'EPS next Q': '0.52', 'Inst Own': '-', 'Short Float': '4.15%', 'Perf Quarter': '-12.33%', 'Sales': '13.24B', 'P/S': '0.21', 'EPS this Y': '298.50%', 'Inst Trans': '-0.38%', 'Short Ratio': '4.85', 'Perf Half Y': '-21.76%', 'Book/sh': '22.39', 'P/B': '1.30', 'ROA': '7.80%', 'Target Price': '40.25', 'Perf Year': '-27.55%', 'Cash/sh': '9.19', 'P/C': '3.17', 'EPS next 5Y': '34.20%', 'ROE': '34.80%', '52W Range From': '27.15', '52W Range To': '50.96', 'Perf YTD': '-39.26%', 'Dividend': '-', 'P/FCF': '-', 'EPS past 5Y': '22.00%', 'ROI': '18.90%', '52W High': '-42.94%', 'Beta': '2.67', 'Div

In [26]:
from finvizfinance.screener.technical import Technical

industry = 'Advertising Agencies'
foverview = Technical()
filters_dict = {'Industry': industry}
foverview.set_filter(filters_dict=filters_dict)
df = foverview.screener_view()
df

,Ticker,Beta,ATR,SMA20,SMA50,SMA200,52W High,52W Low,RSI,Price,Change,from Open,Gap,Volume
0,ADTH,NaN,0.27,-0.2443,-0.3213,-0.6992,-0.8445,0.0054,32.23,1.85,-0.0314,-0.0314,0.0000,42163.0
1,ADTHW,NaN,0.04,-0.3980,-0.4579,-0.7945,-0.9111,-0.1579,34.90,0.16,-0.1579,-0.1111,-0.0526,24304.0
2,ADV,NaN,0.18,-0.2041,-0.3354,-0.5349,-0.7318,0.0649,25.93,2.55,-0.0136,-0.0249,0.0116,270972.0
3,ANTE,1.04,0.08,-0.2517,-0.3626,-0.4772,-0.7235,-0.0121,23.85,0.74,-0.0255,-0.0378,0.0128,8043.0
4,ATY,NaN,0.11,-0.1206,-0.2023,-0.3000,-0.7481,0.1619,30.19,1.92,-0.0556,-0.0693,0.0148,34120.0
5,BAOS,NaN,0.09,-0.1587,-0.2083,-0.4543,-0.8748,0.0520,30.99,1.01,-0.0288,-0.0194,-0.0096,19814.0
6,BOC,0.52,0.97,-0.1091,-0.0916,-0.0781,-0.4809,0.2366,33.24,23.10,-0.0195,-0.0166,-0.0030,32072.0
7,CCO,2.51,0.14,-0.1554,-0.0923,-0.4112,-0.6488,0.5878,38.20,1.44,-0.0828,-0.0769,-0.0064,1015906.0
8,CMPR,1.51,2.29,-0.2313,-0.3105,-0.5334,-0.7496,-0.0196,29.20,24.52,-0.0573,-0.0555,-0.0019,58000.0
9,CNET,1.87,0.11,0.3899,0.3869,0.5584,-0.3436,2.8173,68.10,1.07,0.1138,0.1224,-0.0077,1337149.0


In [61]:
from hashlib import new
from django.db import models
from unicodedata import name
from finvizfinance.screener.overview import Overview
from finvizfinance.screener.valuation import Valuation
from finvizfinance.quote import finvizfinance
import math
import statistics
import time
from urllib.request import Request, urlopen
import regex as re
#from django.apps import AppConfig
from django.conf import settings
from logging.config import IDENTIFIER
from audioop import avg
from pickle import TRUE
from cmath import nan
import pandas as pd




12
['Banks - Regional', 'Biotechnology', 'Shell Companies', 'Asset Management', 'Auto & Truck Dealerships', 'Drug Manufacturers - Specialty & Generic', 'Electronic Components', 'Engineering & Construction', 'Entertainment', 'Insurance - Life', 'Software - Application', 'Software - Infrastructure']


In [59]:
def get_bigindustry_list():
    req = Request("https://finviz.com/groups.ashx?g=industry&v=110&#o=name",headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read().decode('utf-8')
    industries = re.findall('(?<=class="tab-link">).*?(?=</a>)',webpage,re.DOTALL)
    industries = industries[:144]
    big_industries = [industries[13],industries[17],industries[118],industries[8],industries[9],industries[37],industries[40],industries[43],industries[44],industries[62],industries[120],industries[121]]
    small_industries = []
    for industry in industries:
        if industry not in big_industries:
            small_industries.append(industry)
    return big_industries

big_stock_dic={}

def get_big_stock_dic():
    bigindlist = get_industry_list()
    newbigindlist=[]
    for i in list:
        i = i.lower()
        i = i.replace(" ", "")
        i = i.replace("&","")
        i = i.replace("-", "")
        newbigindlist.append(i)

    rlist = [
                '','21','41','61','81','101','121','141','161','181','201','221','241','261','281','301','321','341','361','381','401','421','441','461','481','501','521','541','561','581','601','621','641','661','681','701','721','741','761','781'
            ]
            
    big_ticka_list = []

    def add_tickers_to_list(tickalist):
        try:
            for ticker in tickalist:
                if ticker not in big_ticka_list:
                    big_ticka_list.append(ticker)
                else: continue
        except:
            pass

    def get_tickers_from_page(i,page):
        reqdef1 = Request('https://finviz.com/screener.ashx?f=ind_'+i+'&r='+page, headers={'User-Agent': 'Mozilla/5.0'})
        urlsite = urlopen(reqdef1).read().decode('utf-8')
        tickalist = re.findall('(?<=class="screener-link-primary">).*?(?=</a>)',urlsite, re.DOTALL)
        return add_tickers_to_list(tickalist)
    for i in newbigindlist[4]:
        for page in rlist:
            try:
                get_tickers_from_page(i,page)
                time.sleep(.3)
            except Exception:
                time.sleep(3)
    for stock_ticker in big_ticka_list:
        stock = None
        while stock == None:
            try:
                stock = finvizfinance(stock_ticker).ticker_fundament()
                time.sleep(2)
            except Exception:
                time.sleep(10)
        # Stores the basic stock data as a variable
        sector = stock['Sector']
        # Gets the sector
        ticker = stock_ticker
        # Gets the ticker
        company_name = stock['Company']
        # Gets the company name
        beta = stock['Beta']
        beta = fix_dashes(beta)
        book_sh = (stock['Book/sh'])
        if book_sh == '-':
            book_sh = str(0)
            bvps = 0
        else:
            bvps = float(book_sh)


        shares_outstanding = stock['Shs Outstand']
        if shares_outstanding == '-':
            shares_outstanding=0
        else:
            shares_outstanding = float(stock['Shs Outstand'][:-1]) * float(DOLLAR_MAPPING[stock['Shs Outstand'][-1]])
        
        book_value = bvps * shares_outstanding

        pb = stock['P/B']
        pb = fix_dashes(pb)
        ps = stock['P/S']
        ps = fix_dashes(ps)
        pe = stock['P/E']
        pe = fix_dashes(pe)
        fwdpe = stock['Forward P/E']
        fwdpe = fix_dashes(fwdpe)
        eps = stock['EPS (ttm)']
        eps = fix_dashes(eps)
        mkt_cap_short = stock['Market Cap']
        if mkt_cap_short == '-':
            mkt_cap_short = str(0)
            mkt_cap = 0
        else:
            mkt_cap = float(mkt_cap_short[:-1]) * float(DOLLAR_MAPPING[mkt_cap_short[-1]])
        #gets mkt cap
        revenue_short = stock['Sales']
        if revenue_short == '-':
            revenue_short = str(0)
            revenue = 0
        else:
            revenue = float(revenue_short[:-1]) * float(DOLLAR_MAPPING[revenue_short[-1]])
        # gets the sales
        profit_short = stock['Income']
        if profit_short == '-':
            profit = 0
            profit_short = '0'
        else:
            profit = float(profit_short[:-1]) * float(DOLLAR_MAPPING[profit_short[-1]])
        # gets the net income
        profit_margin = stock['Profit Margin']
        if profit_margin == '-':
            profit_margin = str(0)
            profit_margin_float = 0
        else:
            profit_margin_float = float(profit_margin[:-1])/100
        # gets the PM
        rev_growth = stock['Sales Q/Q']
        if rev_growth == '-':
            rev_growth = str(0)
            rev_growth_float = 0
        else:
            rev_growth_float = float(rev_growth[:-1])/100
        # gets rev growth
        avg_volume = stock['Avg Volume']

        if avg_volume == '-':
            avg_volume = str(0)
            avg_volume_float=0
        else:
            avg_volume_float=float(avg_volume[:-1]) * float(DOLLAR_MAPPING[avg_volume[-1]])

        shares_float = stock['Shs Float']

        if shares_float == '-':
            shares_float = str(0)
            shares_float_float=0
        else:
            shares_float_float=float(shares_float[:-1]) * float(DOLLAR_MAPPING[shares_float[-1]])
        
        short_float = stock['Short Float']

        if short_float == '-':
            short_float = str(0)
            short_float_float = 0
        else:
            short_float_float = float(short_float[:-1])/100

        # gets revenue growth
        big_stock_dic[ticker] = {'company_name': company_name, 'ticker': ticker, 'industry': industry, 'beta': beta, 'mkt_cap' : mkt_cap,'mkt_cap_short': mkt_cap_short, 'revenue': revenue, 'revenue_short':revenue_short,'profit': profit, 'profit_short':profit_short, 'profit_margin':profit_margin, 'sector': sector,'rev_growth': rev_growth, 'pe': pe,'fwdpe':fwdpe, 'profit_margin_float': profit_margin_float,'rev_growth_float':rev_growth_float,'ps':ps,'eps':eps,'pb':pb,'avg_volume':avg_volume,'shares_float':shares_float,'short_float':short_float,'book_value':book_value,'bvps':bvps,'avg_volume_float':avg_volume_float,'shares_float_float':shares_float_float,'short_float_float':short_float_float}
    return big_stock_dic

['ABCB', 'ABTX', 'ACNB', 'AFBI', 'ALRS', 'AMAL', 'AMNB', 'AMTB', 'AROW', 'ASB', 'ASRV', 'ATLO', 'AUB', 'AUBN', 'AVAL', 'AX', 'BANC', 'BANF', 'BANR', 'BAP', 'BBAR', 'BBD', 'BCBP', 'BCH', 'BCML', 'BFC', 'BFIN', 'BFST', 'BHB', 'BHLB', 'BKSC', 'BKU', 'BLFY', 'BLX', 'BMA', 'BMRC', 'BOH', 'BOKF', 'BOTJ', 'BPOP', 'BRKL', 'BSAC', 'BSBK', 'BSBR', 'BSMX', 'BSRR', 'BSVN', 'BUSE', 'BWB', 'BWBBP', 'BWFG', 'BY', 'BYFC', 'CAC', 'CADE', 'CARE', 'CARV', 'CASH', 'CATC', 'CATY', 'CBAN', 'CBFV', 'CBNK', 'CBSH', 'CBTX', 'CBU', 'CCB', 'CCBG', 'CCNE', 'CFB', 'CFBK', 'CFFI', 'CFFN', 'CFG', 'CFR', 'CHCO', 'CHMG', 'CIB', 'CIVB', 'CIZN', 'CLBK', 'CMA', 'CNOB', 'COLB', 'CPF', 'CSTR', 'CTBI', 'CUBI', 'CVBF', 'CVCY', 'CVLY', 'CWBC', 'CZNC', 'CZWI', 'DB', 'DCOM', 'EBC', 'EBMT', 'EBTC', 'ECBK', 'EFSC', 'EGBN', 'EMCF', 'EQBK', 'ESQ', 'ESSA', 'EVBN', 'FBC', 'FBIZ', 'FBK', 'FBMS', 'FBNC', 'FBP', 'FCAP', 'FCBC', 'FCCO', 'FCF', 'FCNCA', 'FCNCO', 'FCNCP', 'FFBC', 'FFBW', 'FFIC', 'FFIN', 'FFNW', 'FFWM', 'FGBI', 'FHB', 'FHN'

In [60]:
print(len(tickerRegList))

2386
